In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [26]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/로또/942회까지로또.xlsx')

data = data.iloc[::-1].reset_index(drop=True)

data['추첨일'] = pd.to_datetime(data['추첨일'])
data['Year'] = data['추첨일'].dt.year
data['Month'] = data['추첨일'].dt.month
data['Day'] = data['추첨일'].dt.day

data = data.drop(['추첨일','회차', 'Bonus', '당첨금4', '당첨금5', '당첨자4', '당첨자5'], axis=1)

data[['당첨금1', '당첨금2', '당첨금3']] = data[['당첨금1', '당첨금2', '당첨금3']].applymap(lambda x: int(x[:-1].replace(',','')))

In [193]:
# 당첨숫자 확률화
prop = tf.keras.utils.to_categorical(data.iloc[:, 6:12]-1).sum(axis=1)/6
prop = pd.DataFrame(prop)

data.drop([0,1,2,3,4,5], axis=1, inplace=True)

data = pd.concat([data, prop], axis=1)

In [210]:
# 시계열 만들기
# selected = [0, 1, 2, 3, 4, 5, 'Year', 'Month', 'Day']

X = []
y = []

step = 10

for i in range(data.shape[0] - step):
  
  X.append(data.values[i:i+step])
  y.append(data[[i for i in range(45)]].values[i+step])

X = np.array(X)
y = np.array(y)
# y = tf.keras.utils.to_categorical(y-1)
# y = y.sum(axis=1)/6

X.shape, y.shape

((932, 10, 54), (932, 45))

In [211]:
# scaling

train_idx = int(X.shape[0] * 0.9)

max_ = X[:train_idx].max()
min_ = X[:train_idx].min()

X = ((X-min_) / (max_-min_))

In [212]:
# train/test split

X_train = X[:train_idx]
y_train = y[:train_idx]

X_val = X[train_idx:-2]
y_val = y[train_idx:-2]

X_test = X[-2:]
y_test = y[-2:]

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(838, 10, 54) (838, 45)
(92, 10, 54) (92, 45)
(2, 10, 54) (2, 45)


In [213]:
def mylottery():

  input_ = tf.keras.Input(shape = X_train.shape[1:])

  layer = tf.keras.layers.GRU(16, return_sequences=True)(input_)
  layer = tf.keras.layers.GRU(16)(layer)

  layer = tf.keras.layers.Dense(32, use_bias=False, kernel_regularizer=tf.keras.regularizers.L2(0.001), kernel_initializer='he_normal')(layer)
  layer = tf.keras.layers.BatchNormalization()(layer)
  layer = tf.keras.layers.Activation('swish')(layer)

  layer = tf.keras.layers.Dense(32, use_bias=False, kernel_regularizer=tf.keras.regularizers.L2(0.001), kernel_initializer='he_normal')(layer)
  layer = tf.keras.layers.BatchNormalization()(layer)
  layer = tf.keras.layers.Activation('swish')(layer)

  output_ = tf.keras.layers.Dense(45, activation='softmax')(layer)


  model = tf.keras.models.Model(input_, output_)

  model.compile(loss='categorical_crossentropy', optimizer='adam')

  return model

In [214]:
model = mylottery()

monitor = 'val_loss'
mode = 'min'

callbacks = [tf.keras.callbacks.EarlyStopping(patience=100, monitor=monitor, mode=mode), 
             tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=50, verbose=1, monitor=monitor, mode=mode), 
             tf.keras.callbacks.ModelCheckpoint('lottery_model.h5', verbose=0, save_best_only=True, monitor=monitor, mode=mode)
             ]


model.fit(X_train, y_train, epochs= 10000, validation_data=(X_val, y_val), callbacks=callbacks)

Epoch 1/10000
27/27 [==============================] - 3s 31ms/step - loss: 4.0013 - val_loss: 3.9322
Epoch 2/10000
27/27 [==============================] - 0s 8ms/step - loss: 3.9478 - val_loss: 3.9271
Epoch 3/10000
27/27 [==============================] - 0s 8ms/step - loss: 3.9347 - val_loss: 3.9225
Epoch 4/10000
27/27 [==============================] - 0s 8ms/step - loss: 3.9199 - val_loss: 3.9181
Epoch 5/10000
27/27 [==============================] - 0s 9ms/step - loss: 3.9118 - val_loss: 3.9143
Epoch 6/10000
27/27 [==============================] - 0s 8ms/step - loss: 3.9078 - val_loss: 3.9108
Epoch 7/10000
27/27 [==============================] - 0s 8ms/step - loss: 3.9005 - val_loss: 3.9070
Epoch 8/10000
27/27 [==============================] - 0s 8ms/step - loss: 3.8951 - val_loss: 3.9031
Epoch 9/10000
27/27 [==============================] - 0s 8ms/step - loss: 3.8899 - val_loss: 3.9010
Epoch 10/10000
27/27 [==============================] - 0s 8ms/step - loss: 3.8883 - val_l

In [215]:
# test 예측
model = tf.keras.models.load_model('lottery_model.h5')
pred = model.predict(X_test)

np.sort(tf.math.top_k(pred, 6).indices + 1)

array([[ 5, 14, 15, 19, 24, 45],
       [ 5,  8, 19, 27, 34, 45]], dtype=int32)

In [216]:
# test 실제
np.sort(tf.math.top_k(y_test, 6).indices + 1)

array([[12, 14, 25, 27, 39, 40],
       [10, 12, 18, 35, 42, 43]], dtype=int32)

In [217]:
# 아무거나 뽑아서 결과 비교
print(np.sort(tf.math.top_k(model.predict(X_val)[23], 6).indices+1))
print(np.sort(tf.math.top_k(y_val[23], 6).indices+1))

[12 15 31 33 34 40]
[ 2  4 30 32 33 43]


In [218]:
# 다음회차 예측

X_next = data.values[-step:][np.newaxis, :, :]

pred = model.predict(X_next)

np.sort(tf.math.top_k(pred, 6).indices+1)

array([[ 4,  5, 18, 20, 34, 35]], dtype=int32)